In [37]:
import numpy as np
import gc

In [38]:
a = []
b = [1,2,3]
c = [4,5,6]

a.append(b)
a.append(c)

del b
del c
gc.collect()



print(a)
print(b)
print(c)

[[1, 2, 3], [4, 5, 6]]


NameError: name 'b' is not defined

In [26]:
# create an empty 3D array with 2 layers, 3 rows and 4 columns
arr = np.empty((2, 3, 4))

# print the array
print(arr)

[[[4.6430201e-310 0.0000000e+000 0.0000000e+000 0.0000000e+000]
  [0.0000000e+000 0.0000000e+000 0.0000000e+000 0.0000000e+000]
  [0.0000000e+000 0.0000000e+000 0.0000000e+000 0.0000000e+000]]

 [[0.0000000e+000 0.0000000e+000 0.0000000e+000 0.0000000e+000]
  [0.0000000e+000 0.0000000e+000 0.0000000e+000 0.0000000e+000]
  [0.0000000e+000 0.0000000e+000 0.0000000e+000 0.0000000e+000]]]


In [23]:
#na = np.array([[1, 2, 3]], dtype=np.float64)
na = np.array()

#na = np.append(na, [[4, 5, 6]], axis=0)


na

array([[1., 2., 3.],
       [4., 5., 6.]])